# 1
> GDL1

- toc: true 
- badges: true
- comments: false
- categories: [jupyter]


Modern neural network (NN) design is built on two algorithmic principles: hierarchical feature learning (this concerns the architecture of the NN), and learning by local gradient-descent driven by backpropagation (concerning the learning dynamics undergone by the NN). 

An instance of training data is modeled as an element of some high-dimensional vector space, making a generic learning problem subject to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). Most tasks of interest are not generic; the data comes from the physical world, and so it inherits low-dimensional structure and symmetries. Requiring an NN to respect the symmetries of the data upon which it acts amounts to a prior.

The notes [BBCV21] construct a blueprint for neural network architecture incorporating these priors, termed _geometric priors_ throughout the notes. Importantly, this blueprint provides a unified perspective of the most successful neural network architectures, at least at the level of the building blocks chosen for the network.

## 1.1 $\quad$ Algebra

_References_

* [Bronstein-Bruna-Cohen-Velicovic 2021](https://arxiv.org/abs/2104.13478), _Geometric deep learning_

* [A first impression of group representations](https://wlou.blog/2018/06/22/a-first-impression-of-group-representations/)

### $\quad\quad$ _<font color='teal'>Graphs, categories, groups</font>_


___

__Def__ $\quad$ A _<font color='purple'>graph</font>_ is a pair $\mathcal{G} = (\mathcal{V}, \mathcal{E})$, where $\mathcal{V}$ is a set whose elements are called _<font color='purple'>vertices</font>_. The set $\mathcal{E}$ consists of _<font color='purple'>edges</font>_, defined to be a multi-set of exactly two vertices in $\mathcal{V}$, not necessarily distinct. 


___

__Def__ $\quad$ A _<font color='purple'>directed graph</font>_ is a pair of sets  $\mathcal{G} = (\mathcal{V}, \mathcal{A})$ of vertices and _arrows_ (or equivalently, _directed edges_). An _<font color='purple'>arrow</font>_ is an ordered pair of vertices.

___

__Def__ $\quad$ Consider an arrow $f$ of a directed graph $\mathcal{G} = ( \mathcal{V}, \mathcal{A})$, specifically $f \equiv (a,b) \in \mathcal{A}$, with $a,b \in \mathcal{V}$. The operations $\textsf{dom}$ and $\textsf{cod}$ act on the arrows $f \in \mathcal{A}$ via 
$$
\textsf{dom}f = a,\, \textsf{cod} f = b\,,
$$ 
and are called the _<font color='purple'>domain</font>_ operation and _<font color='purple'>codomain</font>_ operation,  respectively. 

___

Given arrows, $f$ and $g$ of some directed graph, say that the ordered pair $(g,f)$ is a _<font color='purple'>composable pair</font>_ if $\textsf{dom} g = \textsf{cod} f$. 
Going forward, we express the relations $a = \textsf{dom} f$ and $b = \textsf{cod} f$ equivalently as
$$
f : a \to b \, \quad \text { or  } \, \quad a \xrightarrow[\,]{ f} b  
$$

The next definition formalizes the behavior of a collection of structure-respecting maps between mathematical objects. 

___

__Def__ $\quad$ A _<font color='purple'>category</font>_ is a directed graph $\mathcal{C} = (\mathcal{O},\mathcal{A})$, whose vertices $\mathcal{O}$ we call _<font color='purple'>objects</font>_, such that 

1. For each object $a \in \mathcal{O}$, there is a unique _<font color='purple'>identity</font>_ arrow $\textrm{id}_a \equiv \mathbf{1}_a : a \to a$, defined by the following property: for all arrows $f : b \to a$ and $g : a \to c$, composition with the identity arrow $\mathbf{1}_a $ gives
$$
\mathbf{1}_a \circ f = f \quad \text{ and } \quad g \circ \mathbf{1}_a = g
$$

2. For each composable pair $(g, f)$ of arrows, there is a unique arrow $g \circ f$ called their _<font color='purple'>composite</font>_, with $g \circ f : \textsf{dom} f \to \textsf{cod} g$, such that the composition operation is associative. Namely, for given objects and arrows in the configuration 
$$a \xrightarrow[\,]{ f} b \xrightarrow[\,]{ g} c \xrightarrow[\,]{ k} d \,,
$$
one always has the equality $k \circ (g \circ f) = (k \circ g ) \circ f$.

___

Given a category $\mathcal{C} = (\mathcal{O},\mathcal{A})$, let
$$
\textsf{hom} (b,c) := \{ \, f : f \in \mathcal{A},  \, \textsf{dom} f = b \in \mathcal{O}, \, \textsf{cod} f = c \in \mathcal{O} \, \}
$$
denote the set of arrows from $b$ to $c$. We use the terms _<font color='purple'>morphism</font>_ and arrow interchangeably. 


Groups are collections of symmetries. A _<font color='purple'>group</font>_ $G$ is a category $\mathcal{C} = (\mathcal{O}, \mathcal{A})$ with $\mathcal{O} = \{ o \}$ ( so that we may identify $G$ with the collection of arrows $\mathcal{A}$ ) such that each arrow has a unique inverse: for $g \in \mathcal{A}$, there is an arrow $h$ such that $g \circ h = h \circ g = \mathbf{1}_o$. 
          
Each arrow $g \in \mathcal{A}$ thus has $\textsf{dom} g = \textsf{cod} g = o$. As remarked, the arrows $g \in \mathcal{A}$ correspond to group elements $g \in G$. The categorical interpretation suggests that the group acts on some abstract object $o \in \mathcal{O}$. In the present context, we care how groups act on data, and how this action is represented to a computer. 

### $\quad\quad$ _<font color='teal'>Group representations</font>_

Linear representation theory allows uses linear algebra to study groups. 

Consider a function $\varphi : G \times V \to V$, where $G$ is a group, and where $V$ is a vector space over $\mathbb{R}$. Through this function, we can identify group elements $g$ with self-maps of $V$, namely $\varphi(g, \cdot) : V \to V$. 

When the map $\varphi$ is understood, or left general, we write $g.v$ in place of $\varphi(g,v)$, and we write $(g.)$ in place of $\varphi(g, \cdot)$. 

The "representatives" $(g.)$ of these group elements $g$ can always be composed. If this compositional structure is compatible with the original group operation, we say $\varphi$ is a _<font color='purple'>group action</font>_ on $V$. 

Specifically, $\varphi$ should satisfy $e.v = v$ for all $v \in V$, where $e$ denotes the identity element of $G$, and in general one has $(gh).v  = g.(h.v)$. 

The group action $\varphi$ is _<font color='purple'>$\mathbb{R}$-linear</font>_ if it is compatible with the $\mathbb{R}$-vector space structure on $V$, i.e. additive and homogeneous. Specifically, if for all $v,w \in V$ and all scalars $\lambda \in \mathbb{R}$, one has $g.(v+w) = g.v + g.w$ and $g.(\lambda v) = \lambda g.v$. 

___

__Def__ $\quad$ 
An _<font color='purple'>$\mathbb{R}$-linear representation</font>_ of group $G$ over $\mathbb{R}$-vector space $V$ is an $\mathbb{R}$-linear group action on $V$.

___

The next example illustrates how linear group representations arise naturally when considering group actions on data. 

Input datum are modeled as members of some vector space $\mathcal{X}$, defined with respect to a domain $\Omega$ and a smaller vector space of _<font color='purple'>channels</font>_, $\mathcal{C}$. We may assume all vector spaces discussed to be finite dimensional. 

Specifically, we consider the vector space of $\mathcal{C}$-valued signals over some finite, discrete domain $\Omega$, which perhaps has some graph structure. This vector space is denoted $\mathcal{X}(\Omega, \mathcal{C})$. 

___
___

### $\quad\quad$ _<font color='teal'>Example: shifts of RGB images</font>_


 $\quad$  Consider, for some $n \in \mathbb{N}$, a signal domain $\Omega = \mathbb{T}_n^2$, where $\mathbb{T}_n^2$ denotes the two-dimensional discrete torus of side-length $n$, namely $( \mathbb{Z} / n\mathbb{Z} )^2$. This domain has natural graph as well as group structures. 

If we imagine each vertex of $\Omega$ to be a pixel, we can express an $n \times n$-pixel color (RGB) image as a signal $x : \Omega \to \mathbb{R}^3$, with the first, second and third coordinates of $\mathbb{R}^3$ reporting R, G and B values of a given pixel. 

We make two observations: 

1. As a vector space, $\mathcal{X}(\Omega)$ is isomorphic to $\mathbb{R}^d$, with $d$ typically very large. In the above example, $d = 3n^2$, which is thirty-thousand for a $n \times n \equiv 100 \times 100$ pixel image. 

2. Any group action on $\Omega$ induces a group action on $\mathcal{X}(\Omega)$. 


Expanding on the latter, consider a group action of $G$ on domain $\Omega$. As the torus $\Omega$ already has group structure, it is natural to think of it acting on itself through translations, i.e. we now additionally consider $G = \mathbb{T}_n^2$. 

The action of $G \equiv \mathbb{T}_n^2$ on itself $\Omega \equiv \mathbb{T}_n^2$ induces a $G$-action on $\mathcal{X}(\Omega)$ as follows: for $g \in G$ signal $x \in \mathcal{X}(\Omega)$, the action $(g, x) \mapsto \mathbf{g}.x \in \mathcal{X}(\Omega)$  is defined pointwise at each $u \in \Omega$:
$$
(\mathbf{g}.x)(u) := x(g.\omega),
$$
where the bold $(\mathbf{g}.)$ is used to distinguish the action on signals from the action on the domain. 
___
___

To summarize: any $G$-action on the domain $\Omega$ induces an $\mathbb{R}$-linear representation of $G$ over the vector space of signals on $\Omega$.

___
___

### $\quad\quad$ _<font color='teal'>Example: One-hot encoding</font>_

$\quad$  It seems like standard practice to encode the collection of classes associated to some ML classification problem as an orthonormal basis. These are given ( to the computer ) in the usual coordinate basis 
$$
e_1 \equiv (1, 0, \dots, 0),\, e_2 \equiv (0,1,\dots, 0),\, \dots, \, e_n \equiv (0,\dots, 0,1) \,,
$$
hence the nomenclature _<font color='purple'>one-hot</font>_. In the preceding example, if one considers a one-hot encoding of the vertices of $\mathbb{T}_n^2$, we see that each signal is expressed with respect to this coordinate system, in the sense that $x = \sum_{j=1}^n x_j e_j$. 

This kind of encoding is useful for considering general symmetries of the domain. For instance, if permuting node labels is a relevant symmetry, the action of the symmetric group $\frak{S}_n$ is naturally represented by $n \times n$ permutation matrices.

___
___

The following definition reformulates the notion of a signal.

___

__Def__ $\quad$ A graph $\mathcal{G} = ( \mathcal{V}, \mathcal{E} )$ is said to be equipped with _<font color='purple'>node features</font>_ if for each $v  \in \mathcal{V}$, one has the additional data of an $s$-dimensional vector $x(v) \in \mathbb{R}^s$, called the features of node $v$. 

___


The term 'features' is compatible with the usage in ML, supposing that our input signal has domain some graph $\mathcal{G}$. In this case, we can think of a neural network as a sequence of node-layers built on top of the graph $\mathcal{G}$. 

An input signal endows the first node layer of a NN with features, and the weights of the neural network propagate these through to node features on the nodes of the rest of the network. The features on the last layer of the network can be read off as the output of the NN function. 

## 1.2 $\quad$ Equivariance

_References_

* [Bronstein--Bruna--Cohen--Velicovic 2021](https://arxiv.org/abs/2104.13478), _Geometric deep learning_

* [Maron--Ben-Hamu--Shamir--Lipman 2018](https://arxiv.org/abs/1812.09902), _Invariant and equivariant graph networks_

We henceforth consider groups $G$ acting on some space of signals $\mathcal{X}(\Omega)$ through an action on some fixed signal domain $\Omega$. 

The group action is encoded in a linear representation $\rho$, assumed to be described in a given input coordinate system, just as we would need to specify to a computer. Thus, if $\dim ( \mathcal{X} ) = n$, for each $g \in G$, the object $\rho(g)$ is an $n \times n$ matrix with real entries.  

The learning dynamics occur in the _<font color='purple'>hypothesis space</font>_ $\textsf{H}$, a collection of functions 
$$
\textsf{H} \subset \{\, F : \mathcal{X}(\Omega) \to \mathcal{Y}\, \} ,
$$
where $\mathcal{Y}$ is a vector space of output channels. The point of the learning blueprint is to be able to choose $\textsf{H}$ compatible with the symmetries of input data. The blueprint describes $\textsf{H}$ explicitly, up to hyperparameters such as depth and layer widths. 

A key aspect of this blueprint is that $F \in \textsf{H}$ should be expressed as a composition of functions, most of which are $G$-equivariant. 

The requirement of $G$-equivariance constitutes a geometric prior. From this prior, one can derive the architecture of a generic CNN when $G$ corresponds to translations, and a family of generalizations for other domains and group actions. 

___

__Def__ $\quad$ Let $\rho$ be a representation of group $G$ over $\mathcal{X}(\Omega)$, and let $\rho'$ be a representation of the same group over the $\mathbb{R}$-vector space $\mathcal{Y}$. A function $F: \mathcal{X}(\Omega) \to \mathcal{Y}$ is _<font color='purple'>$G$-equivariant</font>_ if for all $g \in G$ and all $x \in \mathcal{X}(\Omega)$, we have $\rho'(g) F(x) = F ( \rho(g) x )$. 

We say $F$ is _<font color='purple'>$G$-invariant</font>_ if this holds when $\rho'$ is the trivial representation, which is to say $F ( \rho(g) x) = F(x)$ for all $g \in G$ and $x \in \mathcal{X}(\Omega)$.

___
___

### $\quad\quad$ _<font color='teal'>Example: permutation invariance and equivariance in data</font>_

$\quad$ Suppose we are given either a set $\mathcal{V}$, or more generally a graph $\mathcal{G} = ( \mathcal{V}, \mathcal{E} )$, with $\# \mathcal{V} = n$ in either case. As discussed, a signal over $\mathcal{V} = \{ v_1, \dots, v_n \}$ can be thought of as a collection of node features $\{ \, x(v_1), \dots, x(v_n) \, \}$, with $x(v_j) \in \mathbb{R}^s$. 

Let us stack the node features as rows of the $n \times s$ _<font color='purple'>design matrix</font>_

$$
\mathbf{X} = 
\left[ 
\begin{matrix}
x_1\\ 
\vdots\\
x_n
\end{matrix}
\right] ,
$$

which is effectively the same object as signal $x$, provided the vertices are labeled as described. The action of $g \in \mathfrak{S}_n$ on this input data is naturally represented as an $n \times n$ permutation matrix, $\mathbf{P} \equiv \rho(g)$.

One standard way to construct a permutation invariant function $F$ in this setting is through the following recipe: a function $\psi$ is independently applied to every node's features, and $\varphi$ is applied on its sum-aggregated outputs.
$$
    F( \mathbf{X}) = \varphi \left( \, \sum_{j \, = \, 1}^n \psi(x_j) \, \right) .
$$ 
Such a function can be thought of as reporting some "global statistic" of signal $x$.


Equivariance manifests even more naturally. Suppose we want to apply a function $F$ to the signal to "update" the node features to a set of _<font color='purple'>latent</font>_ node features.  

This is the case in which the NN outputs an image segmentation mask; the underlying domain does not change, but the features at each node are updated to the extent that they may not even agree on the number of channels. 

We can stack these latent features into another design matrix, $\mathbf{H} = F(\mathbf{X})$. The order of the rows of $\mathbf{H}$ should clearly be tied to the order of the rows of $\mathbf{X}$, i.e. permutation equivariant: for any permutation matrix $\mathbf{P}$, it holds that $F(\mathbf{P} \mathbf{X} ) = \mathbf{P} F(\mathbf{X})$. 

As a concrete example of a permutation equivariant function, consider a weight matrix $\theta \in \mathbb{R}^{s \, \times \, s'}$. This matrix can be used to map a length-$s$ feature vector at a given node to some new, updated feature vector with $s'$ channels. 

Applying this matrix to each row of the design matrix is an example of a _<font color='purple'>shared node-wise linear transform</font>_, and constitutes a linear, $\mathfrak{S}_n$-equivariant map. Note that, if one wishes to express this map in coordinates, it seems the correct object to consider is a $3$-tensor constructed as a stack of $n$ copies of $\theta$.

___
___

The above example considered signals over the nodes of a graph only, thus label permutation symmetry applies equally well, regardless of the graph structure ( or lack of structure ) underlying such functions. 

In the case that signals $x$ have a domain with graph structure, encoded by adjacency matrix $\mathbf{A}$, it feels right to work with a hypothesis space recognizing this structure.

This is to say that we wish to consider functions $F \in \textsf{H}$ with $F \equiv F( \mathbf{X}, \mathbf{A} )$. Such a function is (label) _<font color='purple'>permutation invariant</font>_ if $F( \mathbf{P} \mathbf{X},\, \mathbf{P} \mathbf{A} \mathbf{P}^{\textrm{T}} ) = F ( \mathbf{X}, \mathbf{A})$, and is _<font color='purple'>permutation equivariant</font>_ if
$$
F( \mathbf{P} \mathbf{X}, \mathbf{P} \mathbf{A} \mathbf{P}^T ) = \mathbf{P}F( \mathbf{X}, \mathbf{A})
$$
for any permutation matrix $\mathbf{P}$. 

___

__Rmk__ $\quad$ One can characterize linear $\mathfrak{S}_n$-equivariant functions on nodes.

> [BBCV 21](https://arxiv.org/abs/2104.13478) : $\quad$ "One can verify any such map can be written as a linear combination of two generators, the identity and the average. As observed by [Maron et al. 2018](https://arxiv.org/abs/1812.09902), any linear permutation equivariant $\mathbf{F}$ can be expressed as a linear combination of fifteen linear generators; remarkably, this family is independent of $n \equiv \# \mathcal{V}$."

___

Amongst the generators discussed just above, the geometric learning blueprint "specifically advocates" for those that are also local, in the sense that the output on node $u$ directly depends on its neighboring nodes in the graph. 


We can formalize this constraint explicitly, by defining the _<font color='purple'>$1$-hop neighborhood</font>_ of node $u$ as
$$
\mathcal{N}(u) := \{ v : \{ u,v \} \in \mathcal{E} \} ,
$$
as well as the corresponding _<font color='purple'>neighborhood features</font>_, 
$$
\mathbf{X}_{\mathcal{N}(u)} := \{ \!\{ \, x_v : v \in \mathcal{N}(u) \, \} \!\} ,
$$
which is a multiset, as indicated by double-brackets, as distinct nodes may be decorated with the same feature vector. 


___
___

### $\quad\quad$ _<font color='teal'>Example: permutations and local averaging</font>_

$\quad$ The node-wise linear transformation described in the previous example can be thought of as operating on $0$-hop neighborhoods of nodes. We generalize this with an example of a function operating on $1$-hop neighborhoods. Instead of a fixed map between feature spaces $\theta : \mathbb{R}^s \to \mathbb{R}^{s'}$, cloned to a pointwise map $\Theta$, we instead specify a _local_ function 
$$
\varphi \equiv \varphi( \, x_u, \, \mathbf{X}_{\mathcal{N}(u)} \, )
$$ 

operating on the features of a node as well as those of its $1$-hop neighborhood. 


We may construct a permutation equivariant function $\Phi$ by applying $\varphi$ to each $1$-hop neighborhood in isolation, and then collecting these into a new feature matrix.

As before, for $\mathcal{V} = \{ v_1, \dots, v_n \}$, we write $x_j$ in place of $x_{v(j)}$, and we similarly write $\mathcal{N}_j$ instead of $\mathcal{N}( v(j) )$.


$$
\Phi ( \mathbf{X}, \mathbf{A} ) = 
\left[
\begin{matrix}
\varphi( \, x_1 , \, \mathbf{X}_{\mathcal{N}_1} \, ) \\
\varphi( \, x_2 , \, \mathbf{X}_{\mathcal{N}_2} \, ) \\
\vdots \\
\varphi( \, x_n , \, \mathbf{X}_{\mathcal{N}_n} \, )
\end{matrix}
\right]
$$

The permutation equivariance of $\Phi$ rests on the output of $\varphi$ being independent of the ordering of the nodes in $\mathcal{N}_u$. Thus, if $\varphi$ is permutation invariant ( a local averaging ) this property is satisfied. 

___
___

> [BBCV 21](https://arxiv.org/abs/2104.13478) : $\quad$ "The choice of $\varphi$ plays a crucial role in the expressive power of the learning scheme."

When considering signals $x \equiv \mathbf{X}$ over graphs, it is natural to consider a hypothesis space whose functions operate on the pair $( \mathbf{X}, \mathbf{A})$, where $\mathbf{A}$ is the adjacency matrix of the signal domain. 

Thus, for such signals the domain naturally becomes part of the input. 

The GDL blueprint distinguishes between two contexts: 

1. one in which the input domain is fixed, and 

2. another in which the input domain varies from signal to signal. 

The preceding example demonstrates that, even in the former context, it can be essential that elements of $\textsf{H}$ treat the (fixed) domain as an argument. 


When the signal domain has geometric structure, it can often be leveraged to construct a _<font color='purple'>coarsening operator</font>_, one of the components of a GDL block in the learning blueprint. Such an operator passes a signal $x \in \mathcal{X}(\Omega)$ to a signal $y \in \mathcal{X}( \Omega')$, where $\Omega'$ is a coarse-grained version of $\Omega$. 

The domain may be fixed for each input, but this domain changes as the signal passes through the layers of the NN, and it is natural that the functions the NN is built out of should pass this data forward. 

> [BBCV 21](https://arxiv.org/abs/2104.13478) : $\quad$ "Due to their additional structure, graphs and grids, unlike sets, can be coarsened in a non-trivial way, giving rise to a variety of pooling operations... more precisely, we cannot define a non-trivial coarsening assuming set structure alone. There exist established approaches that infer topological structure from unordered sets, and those can admit non-trivial coarsening"

## 1.3 $\quad$ Equivariance in neural networks

_References_

* [Kondor-Trivedi 2018](https://arxiv.org/abs/1802.03690)

### $\quad\quad$ _<font color='teal'>Feed-forward neural networks</font>_

The building blocks of neural networks are tensor objects. It is convenient for now to present a definition of a neural network without introducing the notion of a tensor. 

___

__Def__ $\quad$ A _<font color='purple'>feed-forward neural network</font>_ 
$$
(\textsf{N}, \textsf{S}, b, w, a )  
$$ 
is a directed graph $(\textsf{N},\textsf{S})$, called the _<font color='purple'>computation graph</font>_, whose vertices $\textsf{N}$ and edges $\textsf{S}$ are equipped with weights, respectively $b$ and $w$, along with an _<font color='purple'>activation function</font>_ $a$. For simplicity, we assume a single activation function $a$ is used across the network.  The objects should satisfy the following:

* The vertices $\textsf{N}$ of the computation graph, also called _<font color='purple'>neurons</font>_, are partitioned into $L+1$ distinct _<font color='purple'>layers</font>_, for some $L \in \mathbb{N}$. The collection of vertices in layer $\ell$ is denoted $\textsf{N}_\ell$. Layer $\textsf{N}_0$ is called the _<font color='purple'>input layer</font>_, while $\textsf{N}_L$ is the _<font color='purple'>output layer</font>_.

* We write $\mathfrak{n}_i^\ell$ for the $i$th neuron in layer $\ell$. Each neuron is equipped with a _<font color='purple'>bias</font>_ parameter, $b_i^\ell$. The vector of biases for neurons in a given layer $\ell$ is denoted $b^\ell$, and the collection of all bias parameters across all layers is denoted $b$. 

* The edges $\textsf{S}$ of the computation graph, called _<font color='purple'>synapses</font>_, only join neurons in consecutive layers, and importantly, the orientation of each edge always points towards the larger layer. This is why the network is "feed-forward." 

* This structure effectively partitions the synapses into layers, indexed by the largest layer among the two endpoints of the edges. We write $\textsf{S}_\ell$ for the $\ell$th layer of synapses, though we could also write $\textsf{S}_{\ell-1, \ell}$ to emphasize that the edges in $\textsf{S}_\ell$ join neurons in $\textsf{N}_{\ell-1}$ to those in $\textsf{N}_\ell$. The synapse in $\textsf{S}_\ell$ joining neuron $\mathfrak{n}_i^{\ell-1}$ to $\mathfrak{n}_j^\ell$ is denoted $\mathfrak{s}_{ij}^{\ell}$.

* Each synapse $\mathfrak{s}_{ij}^{\ell}$ is equipped with a _<font color='purple'>weight</font>_ parameter, $w_{ij}^{\ell}$. The collection of weights associated to neurons in layer $\ell$ is denoted $w^\ell$, and the collection of all weights across all layers, $w$. 

* For all layers but the input layer, and given an input vector $x$, we also denote the _<font color='purple'>activation at layer $\ell$</font>_ and the _<font color='purple'>activation at neuron $\mathfrak{n}_j^\ell$</font>_ by $a^{\ell}$ and by $a_j^\ell$ respectively. The activations are defined inductively by
$$
a_j^\ell = a \left( \left( \sum_{i} w_{ij}^\ell a_i^{\ell-1} \right) + b_j^{\ell}\right) , 
$$
with the activation at the $0$th layer, $a^0$ is defined as some input signal $x$, without any transformation applied to it.

___

The parameter $L$, controlling the number of layers, is the _<font color='purple'>depth</font>_ of the network, while the _<font color='purple'>width of layer $\ell$</font>_ is the number of neurons in the layer.

We will often abuse notation slightly, and write $\textsf{N}$ in place of the quintuple $(\textsf{N}, \textsf{S}, b, w, a )$. Each feed-forward neural network $\textsf{N}$ of depth $L$ is associated to a function 
$$x \mapsto F_{\textsf{N}}(x) \equiv a^L(\textsf{N},x)$$

The function $F_{\textsf{N}}$ can be expressed as the the alternating composition of nonlinearities and transformations between layers

$$
\mathbf{a} \circ B_L \circ \mathbf{a} \circ B_{L-1} \circ \mathbf{a} \dots \circ  \mathbf{a} \circ B_1,
$$

where $\mathbf{a}$ denotes the nonlinearity $a$ applied entrywise to its vector argument.

In the present subsection, it is convenient to consider each layer of neurons as embedded in $\mathbb{N}$ through some enumeration. Signals in the input layer are thus compactly supported functions $x : \mathbb{N} \to \mathcal{C}$, where $\mathcal{C}$ is the input channel vector space. 

As signal $x$ is propagated forward in the network, its codomain may change to a different channel vector space, so that in general,

$$
B_\ell : \mathcal{X}_c(\mathbb{N}, \mathcal{C}_{\ell-1} ) \to \mathcal{X}_c (\mathbb{N}, \mathcal{C}_\ell ), \quad z \mapsto w^\ell z + b^\ell
$$



We may thus express $F_{\textsf{N}}$ as 

$$ \mathcal{X}_c( \mathbb{N}, \mathcal{C} ) \xrightarrow[\,]{ B_1} \mathcal{X}_c( \mathbb{N}, \mathcal{C}_1 )  \xrightarrow[\,]{ \mathbf{a}} \mathcal{X}_c( \mathbb{N}, \mathcal{C}_1 )  \xrightarrow[\,]{ B_2} \mathcal{X}_c(\mathbb{N}, \mathcal{C}_2) \xrightarrow[\,]{ \mathbf{a} }  \, \dots \, \xrightarrow[\,]{ B_L} \mathcal{X}_c(\mathbb{N}, \mathcal{C}_L) \xrightarrow[\,]{ \mathbf{a} } \mathcal{X}_c(\mathbb{N}, \mathcal{C}_L)
$$

This view obscures the width of each layer, as well as the shape of the signal as it propagates through the layers. Nonetheless, it suffices to discuss equivariance in the context of such functions. Henceforth, we use the term neural network to indicate a feed forward neural network. 

### $\quad\quad$ _<font color='teal'>Equivariant neural networks</font>_

In the present context, we suppose there is a group $G$ acting on each of the signal domains. 

As discussed, the signal domains have been obscured by embedding each domain into $\mathbb{N}$. We should keep in mind that these signal domains are changing, a priori, with each new layer. As such, the group $G$ may act differently on the signal domain at each layer, leading to different representations of $G$, which we wish to distinguish in our notation. 

For $\ell = 0, \dots, L$, let $\rho^\ell$ denote the representation of $G$ at the $\ell$th layer, so that for each $g \in G$, $\rho_g^\ell$ is an invertible linear map,

$$
\rho_g^\ell : \mathcal{X}_c( \mathbb{N} , \mathcal{C}_\ell) \to  \mathcal{X}_c( \mathbb{N} , \mathcal{C}_\ell).
$$

___

__Def__ $\quad$ Let $\textsf{N}$ be a feed-forward neural network of depth $L$. Let $G$ be a group acting on the signal domain of each layer of $\mathbb{N}$. The associated representation of this action on signals, at layer $\ell$, is denoted $\rho^\ell$ as above. The network $\textsf{N}$ is said to be _<font color='purple'>$G$-equivariant</font>_ if each $B_\ell$ is $G$-equivariant, in the sense that for all $g \in G$, and all appropriately supported $z \in \mathcal{X}_c(\mathbb{N}, \mathcal{C}_{\ell-1})$, one has

$$
\rho_g^{\ell} B_\ell (z) = B_\ell (\rho_g^{\ell-1}z ) 
$$


___

The main result of [KT 18](https://arxiv.org/abs/1802.03690) is a classification of equivariant neural networks: they show each $B_\ell$ corresponds to a "generalized" convolutional layer, which we now introduce.

### $\quad\quad$ _<font color='teal'>Convolution</font>_

According to the usual analytic definition, given $\varphi, \theta : \mathbb{R} \to \mathbb{R}$ sufficiently nice, their _<font color='purple'>convolution</font>_ $\varphi \star \theta$ is defined by
$$
( \varphi \star \theta ) (u) = \int_{\mathbb{R}} \varphi(u-v) \theta (v) \, \textrm{d} v
$$
This operation is commutative, and we can interchangeably think of $\varphi$ and $\theta$ as signal and filter. 

This definition can be generalized to (say, $\mathbb{R}$-valued) functions/signals over other domains, in particular compact groups, which admit unique Haar probability measures. 





___

__Def__  $\quad$ Letting $\mu_G$ denote the unique Haar probability measure of compact group $G$, we define the _<font color="purple">convolution</font>_ of two functions $\varphi, \theta : G \to \mathbb{R}$ by
$$
(\varphi \star \theta) (g) = \int_G \varphi(gh^{-1}) \theta(h) \, \textrm{d}\mu_G(h)
$$
___


We have already seen an instance in which the signal domain is a compact group $-$ when modeling images, RGB or grayscale, we took the domain to be the discrete two-dimensional torus $\mathbb{T}_n^2$.

The domains of the signals that propagate through the network aren't necessarily groups. In general, we require these domains to be _<font color="purple">homogeneous spaces</font>_, which means they are equipped with a _<font color="purple">transitive</font>_ $G$-action. Going forward, we also assume that $G$ is some finite group, which is reasonable if $G$ is to be represented to a computer.



Let $G$ be a compact group acting on homogeneous space $\Omega$. Let us fix a distinguished "basepoint" $\omega_* \in \Omega$. 

The basepoint allows us to index the space $\Omega$ using $G$, through the transitivity of its action. Any $\omega \in \Omega$ can be expressed as $g.\omega_\circ$ for some $g \in G$. Let us write
$$
[g]_{\Omega} := g.\omega_\circ
$$
to suppress the dependence of this indexing on the basepoint. 

Observe that group elements of $G$ which fix $\omega_0$ form a subgroup, the _<font color="purple">stabilizer</font>_ of $\omega_\circ$, denoted $H$. The _<font color="purple">(left) quotient space</font>_ $G / H$ is the collection of _<font color="purple">left cosets</font>_  

$$
gH := \{ gh : h \in H \},
$$

and through the map $[ \cdot ]_\Omega$, this quotient space can be identified with the domain $\Omega$ itself. 

For each coset $gH$, we may pick a _<font color="purple">coset representative</font>_ $g' \in gH$. For $\omega \in \Omega$, let $\bar{\omega}$ denote a coset representative of the coset of group elements which map $\omega_\circ$ to $\omega$. 





___

__Def__ $\quad$ Consider a function $\varphi : G \to \mathbb{R}$, and suppose that $G$ acts on some homogeneous space $\Omega$, with stabilizer $H$, so that $\Omega \cong G/H$. The _<font color="purple">projection</font>_ of $\varphi$ to $\Omega$ is the function $\varphi_\downarrow : \Omega \to \mathbb{R}$ given by averaging $\varphi$ over the coset corresponding to a given $\omega \in \Omega$:
$$
\varphi_\downarrow (\omega) := \frac{1}{\# H} \sum_{g \in \bar{\omega}H} f(g).
$$
Conversely, given $\varphi : \Omega \to \mathbb{R}$, the _<font color="purple">lift</font>_ of $\varphi$ to $G$ is the function $\varphi^\uparrow: G \to \mathbb{R}$ given by
$$
\varphi^\uparrow (g) := \varphi([g]_\Omega).
$$
___

The notion of a lift allows us to generalize convolution to pairs of functions whose domains are (perhaps distinct) quotient spaces of the group $G$. Let $\Omega$ and $\Omega'$ be such quotient spaces, and consider $\varphi: \Omega \to \mathbb{R}$ and $\theta : \Omega \to \mathbb{R}$; their _<font color="purple">convolution</font>_ $\varphi \star \theta : G \to \mathbb{R}$ is defined by
$$
(\varphi \star \theta) (g) = \sum_{h \in G} \varphi^\uparrow(gh^{-1}) \theta^\uparrow(h)
$$

The definition of a lift generalizes to functions with domain some right quotient space $H \backslash G$, consisting of right cosets 
$$
Hg := \{ hg : h \in H \},
$$
as well as to functions defined on the double quotient space $H \backslash G / K$, for some subgroups $H, K \subset G$, whose elements are the double cosets
$$
HgK := \{ hgk : h \in H, k \in K \}.
$$

Thus, the definition of convolution can be extended to pairs of functions whose domains are any type of quotient space associated to $G$, and the two functions need not have the same type of domain. 

___

__Def__ $\quad$ Let $G$ be a finite group, and consider two, perhaps distinct, associated homogeneous space $\Omega$ and $\Omega'$. Consider functions $\varphi : \Omega \to \mathbb{R}$ and $\theta : \Omega' \to \mathbb{R}$. The _<font color="purple">generalized convolution</font>_ $\varphi \star \theta : G \to \mathbb{R}$ is 
$$
(\varphi \star \theta) (g) = \sum_{h \in G} \varphi^\uparrow(gh^{-1}) \theta^\uparrow(h). 
$$
___

Now that we have some understanding of the structural requirements we make of signal domains, we are more specific about the output $F_{\textsf{N}}$ of some neural network $\textsf{N}$; it can be expressed as 

$$ \mathcal{X}( \Omega, \mathcal{C} ) \xrightarrow[\,]{ B_1} \mathcal{X}( \Omega_1, \mathcal{C}_1 )  \xrightarrow[\,]{ \mathbf{a}} \mathcal{X}( \Omega_1, \mathcal{C}_1 )  \xrightarrow[\,]{ B_2} \mathcal{X}(\Omega_2, \mathcal{C}_2) \xrightarrow[\,]{ \mathbf{a} }  \, \dots \, \xrightarrow[\,]{ B_L} \mathcal{X}(\Omega_L, \mathcal{C}_L) \xrightarrow[\,]{ \mathbf{a} } \mathcal{X}(\Omega_L, \mathcal{C}_L),
$$

where $\Omega \equiv \Omega_0, \Omega_1, \dots, \Omega_L$ is a sequence of homogeneous spaces associated to $G$. In particular, we may identify each $\Omega_\ell$ with its stabilizer subgroup $H_\ell$ in $G$, so that $\Omega_\ell \cong G / H_\ell$. 

Let us remark that each affine block $B_\ell$ consists of a linear transformation $T_\ell : \mathcal{X}(\Omega_{\ell-1}, \mathcal{C}_{\ell-1}) \to \mathcal{X}(\Omega_\ell, \mathcal{C}_\ell)$, followed by addition of the bias vector $b^\ell$. Previously, $T_\ell$ appeared as the matrix $w^\ell \equiv [ w_{ij}^\ell ]_{i,j}$, but it may be most natural to reshape this into a tensor depending on the context. 




___
__Def__ $\quad$ Let $\textsf{N}$ be a feed-forward neural network of depth $L$, whose layers $\Omega \equiv \Omega_0, \Omega_1, \dots, \Omega_L$ are homogeneous spaces associated to finite group $G$, with each $\Omega_\ell$ in correspondence with its stabilizer in $G$, denoted $H_\ell$. The affine blocks of $\textsf{N}$ are, as usual, denoted $B_\ell$ for $\ell \in \{1, \dots, L\}$, and the linear transformation within each block is denoted $T_\ell$. We call $\textsf{N}$ a _<font color = "purple">$G$-convolutional neural network ($G$-CNN)</font>_ if each $T_\ell$ may be expressed as
$$
T_\ell (z) = z \star \theta_\ell,
$$
for some _<font color="purple">filter</font>_ $$
\theta_\ell \in \mathcal{X} ( H_{\ell-1} \backslash G / H_\ell , \mathbb{R} )
$$



___

## 1.4 $\quad$ Tensors 

Consider a pair of orthonormal bases, the 'original basis' $(e_1, \dots, e_n)$ and the 'new basis' $(\tilde{e}_1, \dots, \tilde{e}_n )$. A signal $x \in \mathbb{R}^n$ can be expressed in either coordinate system 

\begin{align}
x &= e_1 x^1 + \dots + e_nx^n\, \equiv \,e_jx^j \\
x &= \tilde{e}_1 \tilde{x}^1 + \dots + \tilde{e}_n \tilde{x}^n\, \equiv\, \tilde{e}_j  \tilde{x}^j ,
\end{align}


where we have used the Einstein summation convention. We adopt this notation going forward. 

We can express the basis $( \tilde{e}_j )$ in terms of the $( e_j)$ via
$$
\tilde{e}_j = e_i \mathbf{S}^i_j \, ,
$$
where $\mathbf{S}$ is called the _direct transformation matrix_ from original basis to the new. We let $\mathbf{T}$ denote $\mathbf{S}^{-1}$, and observe that the coordinate coefficients transform _contravariantly_:
$$
\tilde{x}^j =  \mathbf{T}_i^j x^i\,.
$$

We assume that the underlying vector space is $\mathbb{R}^n$, and moreover, we equip it with the usual inner product ( in order to make sense of orthogonality ), and note that the Riesz representation theorem allows us to assign a given orthonormal basis $(e_j)$ of vector space $V$ to a canonical dual basis $(e^j)$ on the dual vector space $V^*$. Elements of $V^*$ are called dual vectors, or covectors. This canonical dual basis is given by

\begin{align}
e^j &= \langle e_j, \cdot \rangle \\
&\equiv e_i  \, \delta^{ij} 
\end{align}

and thus,
$$
e_j = \delta_{ij} \, e^i. 
$$

The dual basis can be thought of as a `coordinate selector,' acting on a vector $x \in V$ by 

\begin{align}
e^j(x) &= e^j \, e_k \, x^k \\
&=x^k \, e^j \,e_k  \\
&= \delta_k^j  \, x^k \\
&= x^j.
\end{align}

Let us consider how the objects 
$$
\delta_{ij} \, \equiv \, e_i \, e_j ,\quad  \delta_i^j  \, \equiv\, e_i \, e^k ,\quad  \delta^{ij} \,\equiv \, e^i \, e^j
$$
transform with the bases, i.e. how to express the following new basis objects
$$
\tilde{\delta}_{ij}, \quad \, \tilde{\delta}_i^j,  \quad \, \tilde{\delta}^{ij}
$$
in terms of the corresponding old, via the matrices $\mathbf{S}$ and $\mathbf{T}$.


One has

\begin{align}
\tilde{\delta}_{ij} 
&\equiv \tilde{e}_i\, \tilde{e}_j \\
&= e_{\ell}\, \mathbf{S}^\ell_i  \, e_k  \,\mathbf{S}^k_j \\
&=  \mathbf{S}^\ell_i \, \delta_{\ell k} \, \mathbf{S}^k_j 
\end{align}


Likewise, one has
$$
\tilde{\delta}_i^j \, = \, \mathbf{S}_i^\ell \, \delta_\ell^k \, \mathbf{T}_k^j \quad \text{ and } \quad \tilde{\delta}^{ij} \, = \, \mathbf{T}_\ell^i \, \delta^{\ell k } \, \mathbf{T}_k^j
$$

Given a change of basis $(e_j) \mapsto (\tilde{e}_j)$, the induced transformation $(e^j) \mapsto (\tilde{e}^j)$ is contravariant, which is consistent with the transformations described above:

\begin{align}
\tilde{e}^j  
&\equiv \tilde{e}_i \, \delta^{ij} \\
&= e_\ell \, \mathbf{S}_i^\ell \, \mathbf{T}_\ell^i \, \delta^{\ell k} \, \mathbf{T}_k^j \\
&= e_\ell \,  \delta^{\ell k } \, \mathbf{T}_k^{j} \\
&=  \mathbf{T}_k^j e^k\,
\end{align}

___
__Def__ $\quad$  A _tensor of type_, or _valency_ $(r,s)$ over vector space $V$ is a multilinear map 
$$
\mathbf{A} : \underbrace{V^* \times \, \dots \, \times V^*}_{ k \text{ copies }}  \,\times \, \underbrace{ V \times \, \dots \, \times V }_{ \ell \text{ copies }} \to \mathbb{R}
$$

___


When a basis $(e_j)$ is given, we can express $\mathbf{A}$ with respect to this coordinate system through a total of $n^{r+s}$ scalars, denoted generically by $\mathbf{A}_{ \quad j_1, \, \dots\, , \, j_s} ^ { i_1, \, \dots \, , \, i_r} $. The coordinate indices $i_1, \dots, i_r$ are the _contravariant indices_, while the $j_1, \dots, j_s$ are the _covariant indices_. They are so-named because of the transformation law of the entries under a change of basis $(e_j ) \mapsto (\tilde{e}_j)$ induces the linear transformation

$$
\tilde{\mathbf{A}} _{ \quad j_1, \, \dots\, , \, j_s} ^ { i_1, \, \dots \, , \, i_r} 
=
\mathbf{T}_{k_1}^{i_1} \, \dots \, \mathbf{T}_{k_r}^{i_r} \, \mathbf{A} _{ \quad \ell_1, \, \dots\, , \, \ell_s} ^ { k_1, \, \dots \, , \, k_r} \, \mathbf{S}_{j_1}^{\ell_1}  \, \dots \, \mathbf{S}_{j_s}^{\ell_s} 
$$

Let $\mathcal{T}(r,s)$ denote the vector space of type-$(r,s)$ tensors over $V$. The dimension of this vector space is $n^{r+s}$, i.e. the number of scalar entries in the coordinate representation of such a tensor. 

There are two operations on tensors of interest

* contraction
* outer product


## $1.5$ $\quad$ GDL blueprint (with fixed input domain)

### $\quad$ _Setup_

Our formal treatment of a classification problem requires the following objects: 

* $\Omega$ : A graph with $n$ vertices. This is the domain of the signals considered.

* $\mathcal{X}(\Omega,\mathbb{R}^s)$ :  The space of $s$-channel signals $x : \Omega \to \mathbb{R}^s$. 

* $(e_j)_{j\,=\,1}^{\,n}$ : A 'spatial' coordinate system for scalar-valued signals $\mathcal{X} \to \mathbb{R}$. 

* $(z_c)_{c \,=\,1}^{\,s}$ : A channel coordinate system. 

* $G$ : A group $G$ acting on $\Omega$, via $\Omega \ni u \mapsto g.u \in \Omega$. This induces the `pointwise' action on the space of signals $\mathcal{X}(\Omega,\mathbb{R}^s)$, denoted $x \mapsto \mathbf{g}.x$. 

* $\rho$ : The representation of the induced action of $G$, $x \mapsto \mathbf{g}.x$. 

The basis 
$$
\{ e_j \otimes z_c : 1 \leq j \leq n,\, 1 \leq c \leq s  \}
$$ 

is the one in which signals are expressed to a computer. Given $x \in \mathcal{X}(\Omega,\mathbb{R}^s)$. In terms of this basis, one can write $x$ as
$$
x = \mathbf{X}^{cj} e_j \otimes z_c ,
$$
Note that $\mathbf{X}$ is the so-called design matrix previously introduced. 





> Let $g \in G$. In the basis $e_j \otimes z_c$, we can then identify $\rho(g)$ with a tensor... _(say more)_

### $\quad$ _coarsening operator_

Let $\Omega$ and $\Omega'$ be domains, $G$ a symmetry group over $\Omega$, and write $\Omega' \prec \Omega$ if $\Omega'$ arises from $\Omega$ through some coarsening operator (presumably this coarsening operator needs to commute with the group action).

### $\quad$ _GDL block_

1. _linear $G$-equivariant layer_

$$
B : \mathcal{X}(\Omega, \mathcal{C}) \to \mathcal{X}(\Omega' , \mathcal{C}')
$$ 
such that $B(g.x) = g.B(x)$ for all $g \in G$ and $x \in \mathcal{X}(\Omega, \mathcal{C})$.

2. _nonlinearity_, or _activation function_ 
$$
a : \mathcal{C} \to \mathcal{C}'
$$ 
applied domain-pointwise as $(\mathbf{a}(x))(u) = a(x(u))$.

3. _local pooling operator_ or _coarsening operator_ 
$$
P : \mathcal{X}(\Omega, \mathcal{C}) \to \mathcal{X}(\Omega', \mathcal{C})
$$ 
which gives us our notion $\Omega' \prec \Omega$.

The last ingredient is a global pooling layer applied last, compositionally. 

4. _$G$-invariant layer_, or _global pooling layer_
$$
A : \mathcal{X} (\Omega, \mathcal{C}) \to \mathcal{Y}
$$ 
satisfying $A(g.x) = A(x)$ for all $g \in G$ and $x \in \mathcal{X}(\Omega, \mathcal{C})$. 

### $\quad$ _Hypothesis space_

These objects can be used to define a class of $G$-invariant functions $f: \mathcal{X}(\Omega, \mathcal{C}) \to \mathcal{Y}$ of the form
$$
f = A \circ \mathbf{a}_J \circ B_J \circ P_{J-1} \circ \dots \circ P_1 \circ \mathbf{a}_1 \circ B_1 ,
$$
where the blocks are selected so that the output space of each block matches the input space of the next one. Different blocks may exploit different choices of symmetry groups $G$.

### $\quad$ _Discussion_

Shift-invariance arises naturally in vision and pattern recognition. In this case, the desired function $f \in \textsf{H}$, typically implemented as a CNN, inputs an image and outputs the probability of the image to contain an object from a certain class. It is often reasonably assumed that the classification result should not be affected by the position of the object in the image, i.e., the function $f$ must be shift-invariant.

Multi-layer perceptrons lack this property, a reason why early (1970s) attempts to apply these architectures to pattern recognition problems failed. The development of NN architectures with local weight sharing, as epitomized by CNNs, was among other reasons motivated by the need for shift-invariant object classification. 



A prototypical application requiring shift-equivariance is image segmentation, where the output of $f$ is a pixel-wise image mask. This segmentation mask must follow shifts in the input image. In this example, the domains of the input and output are the same, but since the input has three color channels while the output has \emph{one channel per class}, the representations $(\rho, \mathcal{X}(\Omega, \mathcal{C}) )$ and $(\rho', \mathcal{Y} \equiv \mathcal{X}(\Omega, \mathcal{C}'))$ are somewhat different. 

When $f$ is implemented as a CNN, it may be written as a composition of $L$ functions, where $L$ is determined by the depth and other hyperparameters:
$$
f = f_L \circ f_{L-1} \circ \dots \circ f_2 \circ f_1 .
$$

Examining the individual layer functions making up CNN, one finds they are not shift-invariant in general but rather shift-equivariant. The last function applied, namely $f_L$, is typically a ``global-pooling" function that is genuinely shift-invariant, causing $f$ to be shift-invariant, but to focus on this ignores the structure we will leverage for purposes of expressivity and regularity. 